In [1]:
import sys
from pathlib import Path

print("=== QEPC Experimental Notebook Bootstrap ===")

# Starting from the notebook directory, go up until we find 'qepc_core'
cwd = Path.cwd()
core_root = None

cur = cwd
for _ in range(8):
    if cur.name == "qepc_core":
        core_root = cur
        break
    if cur.parent == cur:
        break
    cur = cur.parent

if core_root is None:
    print("❌ Could not find qepc_core in parent dirs. CWD is:", cwd)
else:
    print("Found qepc_core at:", core_root)
    # Put qepc_core *before* everything else so its qepc package wins
    core_str = str(core_root)
    if core_str not in sys.path:
        sys.path.insert(0, core_str)
        print("➕ Added to sys.path:", core_str)
    else:
        print("ℹ️ qepc_core already in sys.path:", core_str)

    # Optional: also add repo root, but AFTER qepc_core
    repo_root = core_root.parent.parent.parent  # qepc_project
    repo_str = str(repo_root)
    if repo_str not in sys.path:
        sys.path.append(repo_str)
        print("➕ Added repo root to sys.path (tail):", repo_str)

    # Test import
    try:
        import qepc  # this should now be the experimental package
        print("✅ Imported qepc from:", getattr(qepc, "__file__", "<?>"))
    except Exception as e:
        print("❌ Still cannot import qepc:", repr(e))

print("=== End QEPC Experimental Bootstrap ===")


=== QEPC Experimental Notebook Bootstrap ===
Found qepc_core at: C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core
➕ Added to sys.path: C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core
➕ Added repo root to sys.path (tail): C:\Users\wdors\qepc_project
✅ Imported qepc from: C:\Users\wdors\qepc_project\experimental\GTP_REWRITE\qepc_core\qepc\__init__.py
=== End QEPC Experimental Bootstrap ===


In [5]:
from qepc.config import detect_project_root, QEPCConfig
from qepc.nba.data_loaders import load_nba_team_logs
from qepc.core.strengths import TeamStrengthsConfig, compute_team_strengths

# 1) Build QEPC config (defines cfg)
project_root = detect_project_root()
cfg = QEPCConfig.from_project_root(project_root)
print("project_root:", project_root)
print("data/raw:    ", cfg.raw_root)

# 2) Load full multi-season NBA logs
logs = load_nba_team_logs(cfg)
print(f"\nLoaded logs: {len(logs)} rows, {len(logs.columns)} columns")
print("Date range:", logs["gameDate"].min(), "→", logs["gameDate"].max())
print("Seasons:", logs["Season"].min(), "→", logs["Season"].max())

# 3) Configure and compute team strengths
str_cfg = TeamStrengthsConfig(
    lookback_seasons=3,
    min_games=50,
    recency_half_life_games=60,
)

strengths_df = compute_team_strengths(
    game_logs=logs,
    config=str_cfg,
    cutoff_date=None,
    verbose=True,
)

print("\nPreview of strengths table:")
display(strengths_df.head(10))
print("\nTotal teams in strengths_df:", len(strengths_df))


project_root: C:\Users\wdors\qepc_project
data/raw:     C:\Users\wdors\qepc_project\data\raw
[20:02:12] ⧉ QEPC: Using NBA team logs file (NBA_API_QEPC_Format priority): C:\Users\wdors\qepc_project\data\raw\NBA_API_QEPC_Format.csv
[20:02:12] ⧉ QEPC: Loaded raw NBA team logs: 23958 rows, 32 columns
[20:02:12] ⧉ QEPC: NBA team logs canonicalized: 23958 rows, dates 2014-10-28 → 2024-04-14, seasons 2014-15 → 2023-24

Loaded logs: 23958 rows, 32 columns
Date range: 2014-10-28 00:00:00 → 2024-04-14 00:00:00
Seasons: 2014-15 → 2023-24
[20:02:12] ⧉ QEPC: Computing team strengths from seasons 2021–2023 (3 seasons, 7380 team-rows)
[20:02:12] ⧉ QEPC: Computed strengths for 30 teams (min_games=50)
[20:02:12] ⧉ QEPC: Volatility summary: min=10.67, median=12.20, max=13.82

Preview of strengths table:


,Team,SeasonMin,SeasonMax,Games,ORtg,DRtg,Pace,Volatility
0,Atlanta Hawks,2021,2023,246,117.607929,0.0,58.803964,12.060293
1,Boston Celtics,2021,2023,246,119.160959,0.0,59.580480,12.612564
2,Brooklyn Nets,2021,2023,246,110.627805,0.0,55.313903,13.095626
3,Charlotte Hornets,2021,2023,246,108.055841,0.0,54.027921,12.583622
4,Chicago Bulls,2021,2023,246,112.738479,0.0,56.369239,11.478229
5,Cleveland Cavaliers,2021,2023,246,111.924244,0.0,55.962122,10.672978
6,Dallas Mavericks,2021,2023,246,115.907749,0.0,57.953874,13.822265
7,Denver Nuggets,2021,2023,246,114.969921,0.0,57.484960,12.191378
8,Detroit Pistons,2021,2023,246,108.943150,0.0,54.471575,11.309976
9,Golden State Warriors,2021,2023,246,117.432509,0.0,58.716255,11.974297



Total teams in strengths_df: 30


In [6]:
import pandas as pd
from qepc.core.lambda_engine import LambdaConfig, compute_lambda_for_schedule

# tiny test schedule
home_team = strengths_df.iloc[0]["Team"]
away_team = strengths_df.iloc[1]["Team"]
test_schedule = pd.DataFrame([{"Home Team": home_team, "Away Team": away_team}])

lam_cfg = LambdaConfig()
schedule_with_lambda = compute_lambda_for_schedule(test_schedule, strengths_df, lam_cfg, verbose=True)
display(schedule_with_lambda)


[20:03:36] ⧉ QEPC: compute_lambda_for_schedule: 1 games, 1 with full strengths, 0 missing


,Home Team,Away Team,lambda_home,lambda_away,vol_home,vol_away,has_strengths
0,Atlanta Hawks,Boston Celtics,95.588044,94.411956,12.060293,12.612564,True
